In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Model
tf.config.run_functions_eagerly(True)
import os
import glob
import h5py
import pandas as pd
import numpy as np
import os,math,sys,random,random
import matplotlib.pyplot as plt
import seaborn as sn
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor

Mounted at /content/drive


In [ ]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
def correlationMatrix(feat1,feat2):
  mean1 = tf.reduce_mean(feat1,axis=0,keepdims=True)
  mean2 = tf.reduce_mean(feat2,axis=0,keepdims=True)

  norm1 = feat1 - mean1
  norm2 = feat2 - mean2

  covar = tf.matmul(tf.transpose(norm1,perm=[1,0]),norm2)

  sqr1 = tf.square(norm1)
  sqr2 = tf.square(norm2)

  sqr1 = tf.reduce_sum(sqr1,axis=0,keepdims=True)
  sqr2 = tf.reduce_sum(sqr2,axis=0,keepdims=True)

  denom = tf.sqrt(tf.matmul(tf.transpose(sqr1,perm=[1,0]),sqr2))

  correlation = covar/(denom+1e-5)
  return tf.reduce_mean(tf.math.abs(correlation))


def cosineDistance(feat1, feat2):
  norm1 = tf.sqrt(tf.reduce_sum(tf.square(feat1),axis=1,keepdims=True))
  norm2 = tf.sqrt(tf.reduce_sum(tf.square(feat2),axis=1,keepdims=True))

  norm = tf.reduce_sum(feat1*feat2,axis=1,keepdims=True)
  norm = norm / ((norm1*norm2)+1e-5)

  return tf.reduce_mean(norm)

def euclideanDistance(feat1,feat2):
  norm = tf.reduce_sum(tf.square(feat1-feat2),axis=1)
  return tf.reduce_mean(norm)

class CentreLoss(layers.Layer):
  def __init__(self,num_classes,feature_dim,centre_weight):
    super().__init__()
    # self.centres = tf.Variable(initial_value=tf.random.uniform((num_classes,feature_dim)),trainable=True,dtype='float32')
    self.centre_weight=centre_weight

  def forward(self,features,labels):
    labels = tf.cast(labels,dtype='int32')
    cent = tf.gather(self.centres,labels,axis=0)

    self.add_loss(self.weight*tf.reduce_mean(tf.reduce_sum((features-cent)**2,axis=1)))
    return features

In [ ]:
loss = tf.keras.metrics.Mean(name='loss')
acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
NUM_CLASSES = 11
FEAT_DIM = 128

class Trainer(Model):
  def __init__(self,model_list,model,distance_fn):
    super().__init__()
    self.model_list = model_list
    self.model = model
    self.fn = distance_fn

  def train_step(self,data):

    with tf.GradientTape() as tape:
      images,outputs = data
      feats,preds = self.model(images,training=True)

      covarLoss = 0
      classificationLoss = 0

      if len(self.model_list)>0:
        for model in self.model_list:
          tempfeats,_ = model(images,training=False)
          covarLoss+=self.fn(feats,tempfeats)
        covarLoss /= len(self.model_list)

      classificationLoss = tf.keras.losses.SparseCategoricalCrossentropy()(outputs,preds)


      totalLoss = classificationLoss + covarLoss

      grads = tape.gradient(totalLoss,self.trainable_weights)
      self.optimizer.apply_gradients(zip(grads,self.trainable_weights))

      loss.update_state(totalLoss)
      acc.update_state(outputs,preds)

      return {"loss":loss.result(),
            "accuracy":acc.result()}

  def test_step(self,data):
    images,outputs = data

    feats,preds = self.model(images,training=False)

    totalLoss = tf.keras.losses.SparseCategoricalCrossentropy()(outputs,preds)
    loss.update_state(totalLoss)
    acc.update_state(outputs,preds)

    return {"loss":loss.result(),
          "accuracy":acc.result()}

  @property
  def metrics(self):
    return [loss,acc]

eval_loss = tf.keras.metrics.Mean(name='loss')
eval_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

class Evaluator(Model):
  def __init__(self,model_list,mode):
    super().__init__()
    self.model_list = model_list
    self.mode = mode

  def test_step(self,data):
    images,outputs = data

    pred = self.model_list[0](images)[1]

    for i in range(1,len(self.model_list)):
      pred += self.model_list[i](images)[1]

    pred = tf.nn.softmax(pred,axis=1)
    eval_acc.update_state(outputs,pred)

    return {"accuracy":eval_acc.result()}

  @property
  def metrics(self):
    return [eval_acc]

class_loss = tf.keras.metrics.Mean(name='loss')
class_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

class Classifier(Model):
  def __init__(self,model_list,mlp):
    super().__init__()
    self.model_list = model_list
    self.mlp = mlp
    self.concat = layers.Concatenate(axis=1)

  def train_step(self,data):
    with tf.GradientTape() as tape:
      images,outputs = data

      feat,_ = self.model_list[0](images)

      for i in range(1,len(self.model_list)):
        tempfeat,_ = self.model_list[i](images)
        feat = self.concat([feat, tempfeat])

      preds = self.mlp(feat,training=True)
      classificationLoss = tf.keras.losses.SparseCategoricalCrossentropy()(outputs,preds)

      grads = tape.gradient(classificationLoss,self.trainable_weights)
      self.optimizer.apply_gradients(zip(grads,self.trainable_weights))

      class_loss.update_state(classificationLoss)
      class_acc.update_state(outputs,preds)

      return {"loss":class_loss.result(),
            "accuracy":class_acc.result()}

  def test_step(self,data):
    images,outputs = data

    feat,_ = self.model_list[0](images)

    for i in range(1,len(self.model_list)):
      tempfeat,_ = self.model_list[i](images)
      feats = self.concat([feat,tempfeat])

    preds = self.mlp(feats,training=False)
    classificationLoss = tf.keras.losses.SparseCategoricalCrossentropy()(outputs,preds)

    class_loss.update_state(classificationLoss)
    class_acc.update_state(outputs,preds)

    return {"loss":class_loss.result(),
          "accuracy":class_acc.result()}

  def probability(self,data):
    images,outputs = data

    feat,_ = self.model_list[0](images)

    for i in range(1,len(self.model_list)):
      tempfeat,_ = self.model_list[i](images)
      feats = self.concat([feat,tempfeat])

    preds = self.mlp(feats,training=False)

    return preds


  @property
  def metrics(self):
    return [class_loss,class_acc]

In [ ]:
class SnapshotEnsemble(Callback):
    # constructor
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()

    # calculate learning rate for epoch
    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = floor(n_epochs/n_cycles)
        cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (cos(cos_inner) + 1)

    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs={}):
        # calculate learning rate
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        # set learning rate
        backend.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrates.append(lr)

    # save models at the end of each cycle
    def on_epoch_end(self, epoch, logs={}):
        # check if we can save model
        epochs_per_cycle = floor(self.epochs / self.cycles)
        # if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
        #     # save model to file
        #     # filename = "/content/drive/My Drive/MIO TCD Utils/snapshot_xception_centre__model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
        #     # self.model.model.save(filename)
        #     # # self.model.centres = tf.Variable(initial_value=tf.random.uniform((NUM_CLASSES,FEAT_DIM)),trainable=True,dtype='float32')

        #     # temp_model = tf.keras.models.clone_model(self.model.model)
        #     # temp_model.load_weights(filename)
        #     # temp_model.trainable = False
        #     # self.model.model_list.append(temp_model)

            # print("Saved")

sa = SnapshotEnsemble(50, 1, 0.00002)

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ommittal","key":"ab39cb1a93af0b2c186dcf455ca65176"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d yash88600/miotcd-dataset-50000-imagesclassification

 98% 353M/359M [00:09<00:00, 35.2MB/s]
100% 359M/359M [00:09<00:00, 41.3MB/s]


In [ ]:
!ls drive/MyDrive

 archive.zip	     Pretrianed_final	  Trainer_Model_3.h5
 Colab_Notebooks     Trainer_Model_1.h5  'Untitled document.gdoc'
 Pretrained_models   Trainer_Model_2.h5


In [ ]:
! mkdir total
! unzip "/content/drive/MyDrive/archive.zip" -d total

Streaming output truncated to the last 5000 lines.
  inflating: total/train1/work_van/00288068.jpg  
  inflating: total/train1/work_van/00288087.jpg  
  inflating: total/train1/work_van/00288101.jpg  
  inflating: total/train1/work_van/00288112.jpg  
  inflating: total/train1/work_van/00288166.jpg  
  inflating: total/train1/work_van/00288170.jpg  
  inflating: total/train1/work_van/00288223.jpg  
  inflating: total/train1/work_van/00288356.jpg  
  inflating: total/train1/work_van/00288357.jpg  
  inflating: total/train1/work_van/00288358.jpg  
  inflating: total/train1/work_van/00288363.jpg  
  inflating: total/train1/work_van/00288377.jpg  
  inflating: total/train1/work_van/00288430.jpg  
  inflating: total/train1/work_van/00288453.jpg  
  inflating: total/train1/work_van/00288499.jpg  
  inflating: total/train1/work_van/00288506.jpg  
  inflating: total/train1/work_van/00288545.jpg  
  inflating: total/train1/work_van/00288546.jpg  
  inflating: total/train1/work_van/00288666.jpg  

In [ ]:
data_label_dict = {
    'articulated_truck' : 0 ,
    'background' : 1,
    'bicycle' : 2,
    'bus' : 3,
    'car' : 4,
    'motorcycle' : 5,
    'non-motorized_vehicle' : 6,
    'pedestrian' : 7,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

In [ ]:
path_list = []
def get_path(files):
    for path in files:
        path_list.append(path)

In [ ]:
import pathlib
for path in glob.glob('/content/total/train1/*'):
    path = path + '/*'
    get_path(glob.glob(path))

In [ ]:
len(path_list)

52801

In [ ]:
from sklearn.model_selection import KFold
def k_fold(path_files, splits = 5):
    x = KFold(n_splits = splits, shuffle = True , random_state = 69)
    for index_t, index_v in x.split(path_files):
        train = path_files[index_t]
        val = path_files[index_v]
        path_list_train.append(train)
        path_list_val.append(val)
    return path_list_train, path_list_val

In [ ]:
path_list_train, path_list_val = [], []
path_list = np.array(path_list)
random.shuffle(path_list)
path_list_train, path_list_val = k_fold(path_list)

In [ ]:
import cv2
#GETS IMAGE ARRAY OUT GIVEN AN IMAGE PATH
def get_input(path):
    im = cv2.imread(path)
    return(im)

def get_output(path):
    img_class = path.split('/')[-2]
    return data_label_dict[img_class]

#GENERATOR FUNCTION TO PASS THE IMAGES AND LABELS TO model.fit FOR TRAINING
def image_generator(files, batch_size = 8, resize = (256, 256)):

      while True:
          batch_paths  = np.random.choice(a = files,
                                          size = batch_size)
          batch_x = []
          batch_y = []

          for input_path in batch_paths:
              input = get_input(input_path)
              output = get_output(input_path)
              input = cv2.resize(input, resize)
              batch_x.append(input)
              batch_y.append(output)

          batch_x = tf.convert_to_tensor(batch_x)
          batch_x = tf.cast(batch_x,dtype='float32')
          batch_x = tf.keras.applications.xception.preprocess_input(batch_x)
          batch_y = tf.convert_to_tensor(batch_y)
          batch_y = tf.cast(batch_y,dtype='float32')
          # batch_x = np.array(batch_x).astype(np.float32)
          # batch_y = np.array(batch_y)

          yield batch_x, batch_y

In [ ]:
train = image_generator(path_list_train[0])
val = image_generator(path_list_val[0])

In [ ]:
opt2 = tf.keras.optimizers.legacy.Adam(learning_rate=0.00002)

In [ ]:
a = tf.keras.models.load_model('/content/drive/MyDrive/Pretrianed_final/Trainer_Model_1.h5')
b = tf.keras.models.load_model('/content/drive/MyDrive/Pretrianed_final/Trainer_Model_2.h5')
c = tf.keras.models.load_model('/content/drive/MyDrive/Pretrianed_final/Trainer_Model_3.h5')

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

In [ ]:
a.compile(optimizer=opt2, loss = 'sparse_categorical_crossentropy')
b.compile(optimizer=opt2, loss = 'sparse_categorical_crossentropy')
c.compile(optimizer=opt2, loss = 'sparse_categorical_crossentropy')

In [ ]:
# l = []
# t = Trainer(l, a, None)
# t.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy')
# t.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

In [ ]:
# l = []
# t = Trainer(l, b, None)
# t.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy')
# t.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

In [ ]:
# l = []
# t = Trainer(l, c, None)
# t.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy')
# t.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

In [ ]:
a.trainable = False
b.trainable = False
c.trainable = False

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.000001)
# to be run

In [ ]:
l = [a, b]
inp = layers.Input(256)
x = layers.Dense(1024)(inp)
# x = layers.Dense(512)(x)
x = layers.Dense(256)(x)
# x = layers.Dense(128)(x)
x = layers.Dense(64)(x)
# x = layers.Dense(32)(x)
x = layers.Dense(11, activation = "softmax")(x)
mod = Model(inp,x)

classifier1 = Classifier(l, mod)
classifier1.compile(optimizer=opt2)

In [ ]:
l = [a, c]
inp = layers.Input(256)
x = layers.Dense(1024)(inp)
# x = layers.Dense(512)(x)
x = layers.Dense(256)(x)
# x = layers.Dense(128)(x)
x = layers.Dense(64)(x)
# x = layers.Dense(32)(x)
x = layers.Dense(11, activation = "softmax")(x)
mod = Model(inp,x)

classifier2 = Classifier(l, mod)
classifier2.compile(optimizer=opt2)

In [ ]:
l = [b, c]
inp = layers.Input(256)
x = layers.Dense(1024)(inp)
# x = layers.Dense(512)(x)
x = layers.Dense(256)(x)
# x = layers.Dense(128)(x)
x = layers.Dense(64)(x)
# x = layers.Dense(32)(x)
x = layers.Dense(11, activation = "softmax")(x)
mod = Model(inp,x)

classifier3 = Classifier(l, mod)
classifier3.compile(optimizer=opt2)

In [ ]:
classifier1.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])
classifier2.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])
classifier3.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])

<ipython-input-22-0b215bd69a40>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier1.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - 39s 3s/step - loss: 3.9972 - accuracy: 0.1125 - val_loss: 2.9201 - val_accuracy: 0.3000
Epoch 2/20
10/10 [==============================] - 30s 3s/step - loss: 2.5150 - accuracy: 0.3500 - val_loss: 1.8767 - val_accuracy: 0.5000
Epoch 3/20
10/10 [==============================] - 28s 3s/step - loss: 1.6190 - accuracy: 0.4875 - val_loss: 0.9556 - val_accuracy: 0.7750
Epoch 4/20
10/10 [==============================] - 27s 3s/step - loss: 0.9568 - accuracy: 0.7500 - val_loss: 0.6558 - val_accuracy: 0.8250
Epoch 5/20
10/10 [==============================] - 29s 3s/step - loss: 0.6225 - accuracy: 0.8375 - val_loss: 0.3952 - val_accuracy: 0.9250
Epoch 6/20
10/10 [==============================] - 29s 3s/step - loss: 0.5538 - accuracy: 0.8750 - val_loss: 0.4096 - val_accuracy: 0.9250
Epoch 7/20
10/10 [==============================] - 27s 3s/step - loss: 0.5220 - accuracy: 0.8625 - val_loss: 0.5006 - val_accuracy: 0.8750
Epoch 8/20
10/10 [=============

<ipython-input-22-0b215bd69a40>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier2.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])


Epoch 1/20
10/10 [==============================] - 31s 3s/step - loss: 2.2850 - accuracy: 0.3125 - val_loss: 0.9235 - val_accuracy: 0.7750
Epoch 2/20
10/10 [==============================] - 30s 3s/step - loss: 0.9354 - accuracy: 0.7250 - val_loss: 0.4466 - val_accuracy: 0.8750
Epoch 3/20
10/10 [==============================] - 28s 3s/step - loss: 0.4760 - accuracy: 0.8750 - val_loss: 0.2425 - val_accuracy: 0.9000
Epoch 4/20
10/10 [==============================] - 30s 3s/step - loss: 0.1897 - accuracy: 0.9500 - val_loss: 0.2032 - val_accuracy: 0.9500
Epoch 5/20
10/10 [==============================] - 28s 3s/step - loss: 0.2285 - accuracy: 0.9250 - val_loss: 0.1810 - val_accuracy: 0.9250
Epoch 6/20
10/10 [==============================] - 32s 3s/step - loss: 0.1789 - accuracy: 0.9500 - val_loss: 0.1126 - val_accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 28s 3s/step - loss: 0.2316 - accuracy: 0.9500 - val_loss: 0.1541 - val_accuracy: 0.9500
Epoch 8/20
10/10 [==

<ipython-input-22-0b215bd69a40>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier3.fit_generator(train, validation_data = val, validation_steps = 5, epochs = 20, steps_per_epoch = 10, callbacks = [sa])


Epoch 1/20
10/10 [==============================] - 28s 3s/step - loss: 2.6248 - accuracy: 0.3250 - val_loss: 0.8799 - val_accuracy: 0.7000
Epoch 2/20
10/10 [==============================] - 31s 3s/step - loss: 0.7587 - accuracy: 0.8125 - val_loss: 0.3077 - val_accuracy: 0.9500
Epoch 3/20
10/10 [==============================] - 29s 3s/step - loss: 0.1941 - accuracy: 0.9625 - val_loss: 0.2434 - val_accuracy: 0.9500
Epoch 4/20
10/10 [==============================] - 39s 4s/step - loss: 0.1082 - accuracy: 0.9625 - val_loss: 0.1385 - val_accuracy: 0.9500
Epoch 5/20
10/10 [==============================] - 30s 3s/step - loss: 0.0989 - accuracy: 0.9750 - val_loss: 0.0858 - val_accuracy: 0.9750
Epoch 6/20
10/10 [==============================] - 29s 3s/step - loss: 0.0935 - accuracy: 0.9750 - val_loss: 0.4353 - val_accuracy: 0.9000
Epoch 7/20
10/10 [==============================] - 28s 3s/step - loss: 0.1259 - accuracy: 0.9625 - val_loss: 0.4183 - val_accuracy: 0.9250
Epoch 8/20
10/10 [==

In [ ]:
# accuracy using a and b
classifier1.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

330/330 [==============================] - 1176s 4s/step - loss: 0.2203 - accuracy: 0.9438


[0.22032958269119263, 0.9437500238418579]

In [ ]:
# accuracy using a and c
classifier2.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

330/330 [==============================] - 1164s 4s/step - loss: 0.1545 - accuracy: 0.9495


[0.15447072684764862, 0.9495264887809753]

In [ ]:
# accuracy using b and c
classifier3.evaluate(image_generator(path_list_val[0], batch_size=32), steps=330)

330/330 [==============================] - 1156s 4s/step - loss: 0.1480 - accuracy: 0.9562


[0.14797227084636688, 0.9561553001403809]

In [ ]:
# y = next(image_generator(path_list_val[0]))
# sample3 = classifier3.probability(y)
# sample1 = classifier1.probability(y)
# sample2 = classifier2.probability(y)

# for i in range(8):
#   print(np.argmax(sample1[i]))
#   print(np.argmax(sample2[i]))
#   print(np.argmax(sample3[i]))

In [ ]:
test = image_generator(path_list_val[0], 64)
i = 0
cnt = 0
max_iter = len(path_list_val[0])
while(True):
    i = i + 1
    if(i > max_iter/64):
        break
    if(i % 5 == 1):
        print("Completed:", (i*64*100.0)/max_iter, "%")
    x = next(test)
    pred1 = np.array(classifier1.probability(x))
    pred2 = np.array(classifier2.probability(x))
    pred3 = np.array(classifier3.probability(x))

    for j in range(64):
      class1 = np.argmax(pred1[j])
      class2 = np.argmax(pred2[j])
      class3 = np.argmax(pred3[j])

      prob1 = pred1[j].max()
      prob2 = pred2[j].max()
      prob3 = pred3[j].max()

      predictions = {(class1,prob1),(class2,prob2),(class3,prob3)}

      counts = {}

      for prediction,probabaility in predictions:
        if prediction in counts:
          counts[prediction] +=1
        else:
          counts[prediction] = 1

      majority_vote = max(counts,key = counts.get)
      majority_vote_count = counts[majority_vote]

      final_prediction = None
      if majority_vote_count == 1:
        max_probability = float('-inf')
        max_probability_class = None

        for prediction,probability in predictions:
          if probability > max_probability:
            max_probability_class = prediction
            max_probability = probability

        final_prediction = max_probability_class
      else:
        final_prediction = majority_vote


      if(final_prediction == x[1][j]):
            cnt = cnt + 1

print("Accuracy obtained by average ensembling the three classifiers:", (cnt*100.0)/max_iter, "%")

Completed: 0.606003219392103 %
Completed: 3.636019316352618 %
Completed: 6.666035413313133 %
Completed: 9.696051510273648 %
Completed: 12.726067607234164 %
Completed: 15.756083704194678 %
Completed: 18.786099801155192 %
Completed: 21.816115898115708 %
Completed: 24.846131995076224 %
Completed: 27.87614809203674 %
Completed: 30.906164188997256 %
Completed: 33.93618028595777 %
Completed: 36.966196382918284 %
Completed: 39.9962124798788 %
Completed: 43.026228576839316 %
Completed: 46.05624467379983 %
Completed: 49.08626077076035 %
Completed: 52.11627686772086 %
Completed: 55.14629296468137 %
Completed: 58.17630906164189 %
Completed: 61.206325158602404 %
Completed: 64.23634125556292 %
Completed: 67.26635735252343 %
Completed: 70.29637344948395 %
Completed: 73.32638954644446 %
Completed: 76.35640564340498 %
Completed: 79.38642174036549 %
Completed: 82.41643783732601 %
Completed: 85.44645393428652 %
Completed: 88.47647003124705 %
Completed: 91.50648612820756 %
Completed: 94.53650222516806 %


In [ ]:
# 'Classifier' object has no attribute 'model' needs to be resolved by removing snapshot ensemble class and implement just cosine annealing

In [ ]:
# def get_input(path):
#     im = cv2.imread(path)
#     return(im)

# def image_generator_preds(files, batch_size = 1, resize = (256, 256)):

#       while True:
#           batch_paths  = np.random.choice(a = files,
#                                           size = batch_size)
#           batch_x = []

#           for input_path in batch_paths:
#               input = get_input(input_path)
#               # output = get_output(input_path)
#               input = cv2.resize(input, resize)
#               batch_x.append(input)
#               # batch_y.append(output)

#           batch_x = tf.convert_to_tensor(batch_x)
#           batch_x = tf.cast(batch_x,dtype='float32')
#           batch_x = tf.keras.applications.xception.preprocess_input(batch_x)
#           # batch_y = tf.convert_to_tensor(batch_y)
#           # batch_y = tf.cast(batch_y,dtype='float32')
#           # batch_x = np.array(batch_x).astype(np.float32)
#           # batch_y = np.array(batch_y)

#           yield batch_x

In [ ]:
# pred_gen = image_generator_preds(path_list_val[0])

In [ ]:
# path_list_val[0][0]

'/content/total/train1/motorcycle/00318507.jpg'

In [ ]:
# classifier.mlp.predict(pred_gen)